In [1]:
######################################################################################## IMPORTS
import nltk
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.sem.relextract import extract_rels
from os.path import isfile
import json
import ftfy
import glob
import re

######################################################################################## RUN ONLY ONCE
# download the pos tagger since its not included by default
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
# enable only if fresh NLTK or lots of issues
nltk.download('all')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Sverre\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\Sverre\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Sverre\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\Sverre\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\Sverre\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger

True

In [2]:
import requests
import re
import hashlib
from spacy import Language, util
from typing import List
from spacy.tokens import Doc, Span
from transformers import pipeline
import crosslingual_coreference
import spacy
from os.path import isfile
import os
import ftfy
import json
import glob
from tqdm import tqdm
import pandas as pd
import numpy as np

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Sverre\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [3]:
import pandas as pd
import numpy as np

In [4]:
import spacy

In [5]:
"""
        Function to apply preprocessing on a selection of files and store it in a separate folder
        - path: root folder (preprocessed-nltk), 
        - subf desired subfolder: AA or AB, must be passed as a string (e.g. "AA") 
        - files in subfolder AA: e.g. p_n_wiki_00 
        - files in subfolder AB: e.g. p_n_wiki_00
        - start: start number file, e.g. 0-99 (no need to fill in 00, 0 is fine)
        - end: end number file, e.g. 0-99
        - the range is inclusive which means, e.g. with (0, 0) you select & pre-process file wiki_00,
        - with (32, 50) you select file wiki_32 up till wiki_50
""" 
def rel_extraction_mul_files(path, subf=None, start=None, end=None, pattern=re.compile(r'.'), relation_string="test_relation_string", first='PERSON', second='GPE'):
    triplets = []
    print("Computing triplets for relation ", relation_string, "with pattern ", pattern)
    if subf:
        # from file start to end
        for i in range(start, end+1):
            # to match the filename p_n_wiki_00 up till p_n_wiki_09 we add a zero in front of the number from user input if necessary
            if i < 10:
                i = "0" + str(i)
            # construct path to file name that falls within range
            f = path + subf + "/p_n_wiki_{}".format(i) 
            
            # if file exists
            if isfile(f):
                # OPEN FILE, GO THROUGH EACH SEN AND PASS THAT INTO COREF FUNCTION
                file = open(f, 'r', encoding='utf-8')
                doc = json.load(file)
                for k, v in doc.items():
                    v = ftfy.fix_text(v) # FIX ANY ENCODINGS
                    # if paragraph has more than one word
                    if len(v.split(" ")) > 1:
                        use = v
                        # tokenize sentence
                        token = word_tokenize(use)
                        # pos_tag sentence
                        pos = pos_tag(token)
                        # chunk sentence
                        chunked = ne_chunk(pos)                        
                        # (any of 'LOCATION', 'ORGANIZATION', 'PERSON', 'DURATION', 'DATE', 'CARDINAL', 'PERCENT', 'MONEY', 'MEASURE')
                        rels = extract_rels(first, second, chunked, corpus='ace', pattern=pattern, window = 100)
                        for relation in rels:
                            #subjsym and objsym
                            triplets.append((k, relation["subjsym"], relation_string, relation["objsym"] ))
    else:
        for f in glob.glob('preprocessed-nltk/*/*'):
            # OPEN EACH FILE, GO THROUGH EACH PARAGRAPH AND PASS THAT INTO COREF FUNC
            file = open(f, 'r', encoding='utf-8')
            doc = json.load(file)
            for k, v in doc.items():
                v = ftfy.fix_text(v) # FIX ANY ENCODINGS
                # if paragraph has more than one word
                if len(v.split(" ")) > 1:
                    use = v
                    # tokenize sentence
                    token = word_tokenize(use)
                    # pos_tag sentence
                    pos = pos_tag(token)
                    # chunk sentence
                    chunked = ne_chunk(pos)                        
                    # (any of 'LOCATION', 'ORGANIZATION', 'PERSON', 'DURATION', 'DATE', 'CARDINAL', 'PERCENT', 'MONEY', 'MEASURE')
                    rels = extract_rels(first, second, chunked, corpus='ace', pattern=pattern, window = 100)
                    for relation in rels:
                            #subjsym and objsym
                            triplets.append((k, relation[0]["subjsym"], relation_string, relation[0]["objsym"] ))
    print("Finished computing relations")
    return triplets

In [6]:
############################################################################### INITIALIZING VARIABLES
path = "./preprocessed-nltk/"

In [7]:
#rel_extraction_mul_files(path, "AA", 0, 99, pattern=re.compile(r'.*\bbear\b.*\bin\b.*'), first='PERSON', second='GPE') # rel_extraction_mul_files(path)

# Computing results

In [62]:
# load evaluation set
evaluation_df = pd.read_csv('Evaluation_Set_CSV.csv')
# unique indexes in EVAL set
indexes = evaluation_df['PAR ID'].unique()
# unique predicates in EVAL set
predicates = evaluation_df['PRED'].unique()

In [63]:
evaluation_df['PRED'].value_counts()

occupation        265
educated at       156
member of         125
date of birth      84
founded by         80
place of birth     49
date of death      13
spouse             13
Name: PRED, dtype: int64

In [64]:
# [0] = pattern [1] = first [2] = second [3] = relation string
#(any of 'LOCATION', 'ORGANIZATION', 'PERSON', 'DURATION', 'DATE', 'CARDINAL', 'PERCENT', 'MONEY', 'MEASURE')
setting_variations = [(re.compile(r'.*\bwork\b.*\bas\b.*'), "PERSON", "ORGANIZATION", "occupation"), 
                        (re.compile(r'.*\bgraduate\b.*\bfrom\b.*'), "PERSON", "ORGANIZATION", "educated at"), 
                            (re.compile(r'.*\bbear\b.*\bon\b.*'), "PERSON", "DATE", "date of birth"), 
                                (re.compile(r'.*\bbear\b.*\bin\b.*'), "PERSON", "GPE", "place of birth"), 
                                    (re.compile(r'.*\bfound\b.*'), "ORGANIZATION", "PERSON", "founded by"), 
                                        (re.compile(r'.*\bmember\b.*\bof\b.*'), "PERSON", "ORGANIZATION", "member of"), 
                                            (re.compile(r'.*\bdie\b.*\bon\b.*'), "PERSON", "DATE", "date of death"), 
                                                (re.compile(r'.*\bhusband\b.*'), "PERSON", "PERSON", "spouse"),
                                                    (re.compile(r'.*\bwife\b.*'), "PERSON", "PERSON", "spouse")]

In [65]:
# full_relations_list = []
# for setting in setting_variations:
#     full_relations_list = full_relations_list + rel_extraction_mul_files(path, "AA", 0, 99, pattern=setting[0], first=setting[1], second=setting[2], relation_string=setting[3])
#     full_relations_list = full_relations_list + rel_extraction_mul_files(path, "AB", 0, 69, pattern=setting[0], first=setting[1], second=setting[2], relation_string=setting[3])
#     print("The length of the full list is now: ", len(full_relations_list))

In [66]:
import csv
# file = open('NLTK_results.csv', 'w', encoding="utf-8")
# writer = csv.writer(file)
# writer.writerow(['PAR ID', 'SUB', 'PRED' , 'OBJ'])
# for i in range(0, len(full_relations_list)):
#     writer.writerow(full_relations_list[i])
# file.close()

# Evaluation

In [67]:
NLTK_results_csv = pd.read_csv('NLTK_results.csv')

In [68]:
NLTK_results_csv = NLTK_results_csv[NLTK_results_csv["PAR ID"].isin(indexes)]
NLTK_results_csv

,PAR ID,SUB,PRED,OBJ
1,93547-3,arthur_john_robin_gorell_milner,occupation,ferranti
2,184462-1,david_harold_bailey,occupation,nasa_ames
108,51694863-7,neuman,occupation,national_institute
109,51745138-3,berlin,occupation,principal_data
110,51947774-7,cynthia_barnhart,occupation,school
...,...,...,...,...
2945,51956447-4,prakash_vijayanath,member of,road
2946,52037349-1,tat_tong,member of,american_idol
2947,52041825-1,tina_l_podlodowski,member of,seattle_city_council
3099,93547-4,arthur_john_robin_gorell_milner,spouse,lucy


In [69]:
indexes_results = NLTK_results_csv['PAR ID'].unique()
# unique predicates in EVAL set
predicates_results = NLTK_results_csv['PRED'].unique()

In [70]:
NLTK_results_csv = NLTK_results_csv.set_index('PAR ID')


In [71]:
evaluation_df = evaluation_df.set_index('PAR ID')

In [72]:
eval_list_true_id = np.vstack({tuple(row) for row in evaluation_df.to_numpy()})

C:\Users\Sverre\AppData\Local\Temp\ipykernel_3992\1994584447.py:1: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  eval_list_true_id = np.vstack({tuple(row) for row in evaluation_df.to_numpy()})


In [73]:
eval_list_pred_id = np.vstack({tuple(row) for row in NLTK_results_csv.to_numpy()})

C:\Users\Sverre\AppData\Local\Temp\ipykernel_3992\2744041742.py:1: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  eval_list_pred_id = np.vstack({tuple(row) for row in NLTK_results_csv.to_numpy()})


In [74]:
dicty = {}
for i in eval_list_pred_id:
    if not i[1] in dicty.keys():
        dicty[i[1]] = 1
    else:
        dicty[i[1]] = dicty[i[1]] + 1
dicty

{'educated at': 4,
 'member of': 10,
 'place of birth': 44,
 'occupation': 8,
 'spouse': 2,
 'founded by': 2}

# Method 1: substring

In [75]:
new_true = [[str(element.replace("_", " ").lower()) for element in triple] for triple in eval_list_true_id]
new_pred = [[str(element.replace("_", " ").lower()) for element in triple] for triple in eval_list_pred_id]

In [76]:
correct = 0
for triple_pred in new_pred:
    for triple_true in new_true:
        if (triple_pred[0] in triple_true[0] or triple_true[0] in triple_pred[0]) and (triple_pred[1] in triple_true[1] or triple_true[1] in triple_pred[1]) and (triple_pred[2] in triple_true[2] or triple_true[2] in triple_pred[2]):
            #print(triple_true)
            #print(triple_pred)
            #print()
            correct += 1

recall = correct/max(1,len(new_true))
precision = correct/max(1,len(new_pred))

In [77]:
print('Number of correct predictions: {}'.format(correct), 'Number of evaluation triples: {}'.format(len(new_true)))

print('Recall: {}'.format(recall))
print('Precision: {}'.format(precision))
print('F1: {}'.format(2*precision*recall/max(1,precision+recall)))

Number of correct predictions: 34 Number of evaluation triples: 779
Recall: 0.043645699614890884
Precision: 0.4857142857142857
F1: 0.042398679625894005


In [78]:
missing_triples = []
for triple_true in new_true:
    triple_missing = True
    for triple_pred in new_pred:
        if (triple_pred[0] in triple_true[0] or triple_true[0] in triple_pred[0]) and (triple_pred[1] in triple_true[1] or triple_true[1] in triple_pred[1]) and (triple_pred[2] in triple_true[2] or triple_true[2] in triple_pred[2]):
            triple_missing = False
    if triple_missing:
        missing_triples.append(triple_true)

print('Number of articles not predicted by NLTK, but in Evaluation set: {}'.format(len(missing_triples)))
#missing_triples

Number of articles not predicted by NLTK, but in Evaluation set: 749


In [79]:
missing_triples = []

for triple_pred in new_pred:
    triple_missing = True
    for triple_true in new_true:
        if (triple_pred[0] in triple_true[0] or triple_true[0] in triple_pred[0]) and (triple_pred[1] in triple_true[1] or triple_true[1] in triple_pred[1]) and (triple_pred[2] in triple_true[2] or triple_true[2] in triple_pred[2]):
            triple_missing = False
    if triple_missing:
        missing_triples.append(triple_pred)

print('Number of articles predicted by NLTK, but not in Evaluation set: {}'.format(len(missing_triples)))
#missing_triples

Number of articles predicted by NLTK, but not in Evaluation set: 37


In [80]:
predicates = ['date of birth', 'occupation', 'educated at', 'spouse', 'place of birth', 'member of', 'founded by', 'date of death']
recall_per_predicate = dict()
precision_per_predicate = dict()
f1_per_predicate = dict()

for predicate in predicates:
    
    true_predicate = []
    pred_predicate = []
    
    for triple in eval_list_true_id:
        if triple[1] == predicate:
            true_predicate.append(triple)
    
    for triple in eval_list_pred_id:
        if triple[1] == predicate:
            pred_predicate.append(triple)

    new_true = [[str(element.replace("_", " ").lower()) for element in triple] for triple in true_predicate]
    new_pred = [[str(element.replace("_", " ").lower()) for element in triple] for triple in pred_predicate]
    print(predicate, len(new_true), len(new_pred))


    correct = 0
    for triple_pred in new_pred:
        for triple_true in new_true:
            # if (triple_pred[0] in triple_true[0] or triple_true[0] in triple_pred[0]) or (triple_pred[2] in triple_true[2] or triple_true[2] in triple_pred[2]):
            #     print("predicted: ", triple_pred, "true: ", triple_true)
            if (triple_pred[0] in triple_true[0] or triple_true[0] in triple_pred[0]) and (triple_pred[1] in triple_true[1] or triple_true[1] in triple_pred[1]) and (triple_pred[2] in triple_true[2] or triple_true[2] in triple_pred[2]):
                correct += 1

    recall = correct/max(1,len(new_true))
    recall_per_predicate[predicate] = recall
    
    precision = correct/max(1,len(new_pred))
    precision_per_predicate[predicate] = precision
    
    f1 = 2*precision*recall/max(1,precision+recall)
    f1_per_predicate[predicate] = f1

date of birth 83 0
occupation 264 8
educated at 155 4
spouse 13 2
place of birth 49 44
member of 124 10
founded by 78 2
date of death 13 0


In [81]:
print('Recall per predicate')
recall_per_predicate

Recall per predicate


{'date of birth': 0.0,
 'occupation': 0.003787878787878788,
 'educated at': 0.03225806451612903,
 'spouse': 0.07692307692307693,
 'place of birth': 0.4897959183673469,
 'member of': 0.024193548387096774,
 'founded by': 0.0,
 'date of death': 0.0}

In [82]:
print('Precision per predicate')
precision_per_predicate

Precision per predicate


{'date of birth': 0.0,
 'occupation': 0.125,
 'educated at': 1.25,
 'spouse': 0.5,
 'place of birth': 0.5454545454545454,
 'member of': 0.3,
 'founded by': 0.0,
 'date of death': 0.0}

In [83]:
print('F1 per predicate')
f1_per_predicate

F1 per predicate


{'date of birth': 0.0,
 'occupation': 0.000946969696969697,
 'educated at': 0.06289308176100629,
 'spouse': 0.07692307692307693,
 'place of birth': 0.5161290322580645,
 'member of': 0.014516129032258063,
 'founded by': 0.0,
 'date of death': 0.0}

# Method 2: similarity

In [84]:
nlp = spacy.load('en_core_web_sm')
new_true = [[nlp(str(element.replace("_", " ").lower())) for element in triple] for triple in eval_list_true_id]
new_pred = [[nlp(str(element.replace("_", " ").lower())) for element in triple] for triple in eval_list_pred_id]

In [85]:
correct = 0
for triple_pred in new_pred:
    for triple_true in new_true:
        if triple_pred[0].similarity(triple_true[0]) >= 0.9 and triple_pred[1].similarity(triple_true[1]) >= 0.99 and triple_pred[2].similarity(triple_true[2]) >= 0.9:
            # print(triple_true)
            # print(triple_pred)
            # print()
            correct += 1

recall = correct/max(1,len(new_true))
precision = correct/max(1,len(new_pred))

In [86]:
print('Number of correct predictions: {}'.format(correct), 'Number of evaluation triples: {}'.format(len(new_true)))

print('Recall: {}'.format(recall))
print('Precision: {}'.format(precision))
print('F1: {}'.format(2*precision*recall/max(1,precision+recall)))

Number of correct predictions: 6 Number of evaluation triples: 779
Recall: 0.007702182284980745
Precision: 0.08571428571428572
F1: 0.001320374105996699


In [87]:
missing_triples = []
for triple_true in new_true:
    triple_missing = True
    for triple_pred in new_pred:
        if triple_pred[0].similarity(triple_true[0]) >= 0.9 and triple_pred[1].similarity(triple_true[1]) >= 0.99 and triple_pred[2].similarity(triple_true[2]) >= 0.9:
            triple_missing = False
    if triple_missing:
        missing_triples.append(triple_true)

print('Number of articles not predicted by NLTK, but in Evaluation set: {}'.format(len(missing_triples)))
#missing_triples

Number of articles not predicted by NLTK, but in Evaluation set: 773


In [88]:
missing_triples = []

for triple_pred in new_pred:
    triple_missing = True
    for triple_true in new_true:
        if triple_pred[0].similarity(triple_true[0]) >= 0.9 and triple_pred[1].similarity(triple_true[1]) >= 0.99 and triple_pred[2].similarity(triple_true[2]) >= 0.9:
            triple_missing = False
    if triple_missing:
        missing_triples.append(triple_pred)

print('Number of articles predicted by NLTK, but not in Evaluation set: {}'.format(len(missing_triples)))
#missing_triples

Number of articles predicted by NLTK, but not in Evaluation set: 64


In [89]:
predicates = ['date of birth', 'occupation', 'educated at', 'spouse', 'place of birth', 'member of', 'founded by', 'date of death']
recall_per_predicate = dict()
precision_per_predicate = dict()
f1_per_predicate = dict()

for predicate in predicates:
    
    true_predicate = []
    pred_predicate = []
    
    for triple in eval_list_true_id:
        if triple[1] == predicate:
            true_predicate.append(triple)
    
    for triple in eval_list_pred_id:
        if triple[1] == predicate:
            pred_predicate.append(triple)

    nlp = spacy.load('en_core_web_sm')
    new_true = [[nlp(str(element.replace("_", " ").lower())) for element in triple] for triple in true_predicate]
    new_pred = [[nlp(str(element.replace("_", " ").lower())) for element in triple] for triple in pred_predicate]

    correct = 0
    for triple_pred in new_pred:
        for triple_true in new_true:
            if triple_pred[0].similarity(triple_true[0]) >= 0.9 and triple_pred[1].similarity(triple_true[1]) >= 0.99 and triple_pred[2].similarity(triple_true[2]) >= 0.9:
                correct += 1

    recall = correct/max(1,len(new_true))
    recall_per_predicate[predicate] = recall
    
    precision = correct/max(1,len(new_pred))
    precision_per_predicate[predicate] = precision
    
    f1 = 2*precision*recall/max(1,precision+recall)
    f1_per_predicate[predicate] = f1

In [90]:
print('Recall per predicate')
recall_per_predicate

Recall per predicate


{'date of birth': 0.0,
 'occupation': 0.003787878787878788,
 'educated at': 0.0,
 'spouse': 0.07692307692307693,
 'place of birth': 0.08163265306122448,
 'member of': 0.0,
 'founded by': 0.0,
 'date of death': 0.0}

In [91]:
print('Precision per predicate')
precision_per_predicate

Precision per predicate


{'date of birth': 0.0,
 'occupation': 0.125,
 'educated at': 0.0,
 'spouse': 0.5,
 'place of birth': 0.09090909090909091,
 'member of': 0.0,
 'founded by': 0.0,
 'date of death': 0.0}

In [92]:
print('F1 per predicate')
f1_per_predicate

F1 per predicate


{'date of birth': 0.0,
 'occupation': 0.000946969696969697,
 'educated at': 0.0,
 'spouse': 0.07692307692307693,
 'place of birth': 0.01484230055658627,
 'member of': 0.0,
 'founded by': 0.0,
 'date of death': 0.0}